In [2]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples/

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-g5gdzyvj
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-g5gdzyvj
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=67e4f7815ee22338d51ae83a63fdbf8af079c1b598a19ba2d6d769d7697b933e
  Stored in directory: /tmp/pip-ephem-wheel-cache-f6_d_7s3/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
/bin/bash: line 1: cuda-install-s

In [94]:
%%cuda --name curand.cu
#include <cstdlib>
#include <stdlib.h>
#include <iostream>
#include <curand.h>
#include <cublas_v2.h>
#include <time.h>
#include <cmath>

using namespace std;

#define kernel kernel_gpu_matmul

// C(m,n) = A(m,k) * B(k,n)
//Умножение матриц на GPU
__global__ void kernel_gpu_matmul(const float *A, const float *B, float *C, const int m, const int k, const int n) {
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    float temp = 0;
    if (row < m && col < k) {
        for (int i = 0; i < n; ++i)
            temp += A[row * n + i] * B[i * n + col];
        C[row * k + col] = temp;
    }
}
//Умножение матриц на GPU при помощи CUBLAS
void gpu_matmul_cublas(const float *A, const float *B, float *C, const int m, const int k, const int n) {
    int lda=m,ldb=n,ldc=k;
    const float alf = 1;
    const float bet = 0;
    const float *alpha = &alf;
    const float *beta = &bet;
    // Create a handle for CUBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);
    // Do the actual multiplication
    cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_T, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);
    // Destroy the handle
    cublasDestroy(handle);
}
//Умножение матриц на CPU
void cpu_matmul(const float* A, const float* B, float* C, const int m, const int k, const int n) {
        for (int i = 0; i < m; ++i)
          for (int j = 0; j < n; ++j)
            for (int r = 0; r < k; ++r)
                C[i * n + j] += A[i * k + r] * B[r * n + j];
}
//Проверка корректности перемножения
void check_matmul(const float* C, const float* C2, const int m, const int n) {
    for (int i = 0; i < m * n; ++i)
        if (abs(C[i] - C2[i]) > 0.001) {
            cout << "The matrices are not equal\n";
            return;
        }
    cout << "The matrices are equal\n";
}
//Вывод матрицы
void print_matrix(float *h_C, int nr_rows_C, int nr_cols_C) {
    for (int i = 0; i < nr_rows_C; ++i){
      for (int j = 0; j < nr_cols_C; ++j)
        cout << *(h_C + i + j * nr_cols_C) << " ";
      cout << endl;
    }
}
//Транспонирование матрицы
void transpose(float* C, const int m, const int n) {
    float *C2 = (float *)malloc(m * n * sizeof(float));
    for (int i = 0; i < m; ++i)
          for (int j = 0; j < n; ++j)
              C2[i * n + j] = C[j * m + i];
    for (int i = 0; i < m * n; ++i)
        C[i] = C2[i];
    free(C2);
}

int main() {
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
    nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = nr_rows_C = nr_cols_C = 300;
    float *h_A = (float *)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float *h_B = (float *)malloc(nr_rows_B * nr_cols_B * sizeof(float));
    float *h_C = (float *)malloc(nr_rows_C * nr_cols_C * sizeof(float));
    float *h_C2 = (float *)malloc(nr_rows_C * nr_cols_C * sizeof(float));
    float *h_C3 = (float *)malloc(nr_rows_C * nr_cols_C * sizeof(float));
    float *d_A, *d_B, *d_C, *d_C3;

    printf("Размерность матриц с типом float: %d (c T, c T)\n", nr_rows_A);
    for (int i = 0; i < nr_rows_A * nr_rows_A; ++i) {
        h_A[i] = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
        h_B[i] = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
        h_C3[i] = 0;
    }
    cudaMalloc(&d_A,nr_rows_A * nr_cols_A * sizeof(float));
    cudaMalloc(&d_B,nr_rows_B * nr_cols_B * sizeof(float));
    cudaMalloc(&d_C,nr_rows_C * nr_cols_C * sizeof(float));
    cudaMalloc(&d_C3,nr_rows_C * nr_cols_C * sizeof(float));
    cudaMemcpy(d_A,h_A,nr_rows_A * nr_cols_A * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_B,h_B,nr_rows_B * nr_cols_B * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_C3,h_C3,nr_rows_B * nr_cols_B * sizeof(float),cudaMemcpyHostToDevice);

//gpu_matmul_cublas
    srand(time(0));
    clock_t start, end;
    start = clock();
    gpu_matmul_cublas(d_A, d_B, d_C, nr_rows_A, nr_cols_A, nr_cols_B);
    end = clock();
    double gpu_time_cublas = static_cast <double>(end - start) / static_cast <double>(CLOCKS_PER_SEC);

    cudaMemcpy(h_C,d_C,nr_rows_C * nr_cols_C * sizeof(float),cudaMemcpyDeviceToHost);
    //Транспонирование матрицы, тк cublas возвращает результат по столбцам
    transpose(h_C, nr_rows_C, nr_cols_C);
    //cout << "C =" << endl;
    //print_matrix(h_C, nr_rows_C, nr_cols_C);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

//cpu_matmul
    clock_t start1, end1;
    start1 = clock();
    cpu_matmul(h_A, h_B, h_C2, nr_rows_A, nr_cols_A, nr_cols_B);
    end1 = clock();
    double cpu_time = static_cast <double>(end1 - start1) / static_cast <double>(CLOCKS_PER_SEC);
    //cout << "C2 =" << endl;
    //print_matrix(h_C2, nr_rows_C, nr_cols_C);

//kernel_gpu_matmul
    cudaEvent_t begin, stop;
    cudaEventCreate(&begin);
    cudaEventCreate(&stop);
    dim3 block_dim(32, 32);
    dim3 grid_dim(ceil(static_cast <double> (nr_rows_A) / static_cast <double> (block_dim.x)), ceil(static_cast <double> (nr_cols_A) / static_cast <double> (block_dim.y)));

    cudaEventRecord(begin, 0);
    kernel_gpu_matmul << <grid_dim, block_dim>> > (d_A, d_B, d_C3, nr_rows_A, nr_cols_A, nr_cols_B);
    cudaEventRecord(stop, 0);

    cudaEventSynchronize(stop);
    float gpu_time;
    cudaEventElapsedTime(&gpu_time, begin, stop);
    cudaMemcpy(h_C3,d_C3,nr_rows_C * nr_cols_C * sizeof(float),cudaMemcpyDeviceToHost);

 //check_matmul
    check_matmul(h_C, h_C2, nr_rows_C, nr_cols_C);
    check_matmul(h_C2, h_C3, nr_rows_C, nr_cols_C);

    cout << "CPU Time: " << cpu_time << endl;
    cout << "GPU(CUBLAS) Time: " << gpu_time_cublas << endl;
    cout << "GPU Time: " << gpu_time << endl;

    free(h_A);
    free(h_B);
    free(h_C);
    free(h_C2);
    free(h_C3);
    cudaFree(d_C3);

    cudaEventDestroy(begin);
    cudaEventDestroy(stop);

    return 0;
}


'File written in /content/src/curand.cu'

In [95]:
!nvcc -o /content/src/curand /content/src/curand.cu -lcurand -lcublas

In [97]:
!/content/src/curand

Размерность матриц с типом float: 300 (c T, c T)
The matrices are equal
The matrices are equal
CPU Time: 0.130083
GPU(CUBLAS) Time: 0.568182
GPU Time: 0.253312
